In [2]:
# import pandas as pd

In [2]:
# data_all = []
# for i in range(4):
#     data = pd.read_csv('/home/aprosvetov/netflix/combined_data_' + str(i+1)+'.txt', header = None, names = ['Cust_Id', 'Rating'], usecols = [0,1])
#     data_all.append(data)

In [3]:
# df = pd.concat(data_all)

In [4]:
# import numpy as np
# df.index = np.arange(0,len(df))
# print('Full dataset shape: {}'.format(df.shape))
# print('-Dataset examples-')
# print(df.iloc[::5000000, :])

Full dataset shape: (100498277, 2)
-Dataset examples-
           Cust_Id  Rating
0               1:     NaN
5000000    2560324     4.0
10000000   2271935     2.0
15000000   1921803     2.0
20000000   1933327     3.0
25000000   1465002     3.0
30000000    961023     4.0
35000000   1372532     5.0
40000000    854274     5.0
45000000    116334     3.0
50000000    768483     3.0
55000000   1331144     5.0
60000000   1609324     2.0
65000000   1699240     3.0
70000000   1776418     4.0
75000000   1643826     5.0
80000000    932047     4.0
85000000   2292868     4.0
90000000    932191     4.0
95000000   1815101     3.0
100000000   872339     4.0


In [ ]:
# import numpy as np
# df_nan = pd.DataFrame(pd.isnull(df.Rating))
# df_nan = df_nan[df_nan['Rating'] == True]
# df_nan = df_nan.reset_index()

# movie_np = []
# movie_id = 1

# for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
#     # numpy approach
#     temp = np.full((1,np.abs(i-j-1)), movie_id)
#     movie_np = np.append(movie_np, temp)
#     movie_id += 1

# # Account for last record and corresponding length
# # numpy approach
# last_record = np.full((1,len(df) - df_nan.iloc[-1, 0] - 1),movie_id)
# movie_np = np.append(movie_np, last_record)

In [ ]:
# df = df[pd.notnull(df['Rating'])]

# df['Movie_Id'] = movie_np.astype(int)
# df['Cust_Id'] = df['Cust_Id'].astype(int)
# print('-Dataset examples-')
# print(df.sample(3))

In [ ]:
# df.to_csv('/srv/aprosvetov/netflix/data_prep.csv', sep=';', index = None)

In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('data_prep.csv', sep=';')

In [2]:
df.head(2)

,Cust_Id,Rating,Movie_Id
0,1488844,3.0,1
1,822109,5.0,1


In [3]:
df.Movie_Id.nunique()

17770

In [4]:
df.Cust_Id.nunique()

480189

In [5]:
cust_sample = df.Cust_Id.sample(10000)

In [6]:
movie_sample = df.Movie_Id.sample(5000)

In [7]:
import surprise

In [8]:
from surprise import Reader, Dataset

In [9]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[df.Cust_Id.isin(cust_sample) &
                              df.Movie_Id.isin(movie_sample)][['Cust_Id', 'Movie_Id', 'Rating']], reader)

In [10]:
from surprise import KNNBasic

In [11]:
sim_options = {
    'name': 'cosine',
    'user_based': False
}

In [12]:
knn = KNNBasic(sim_options=sim_options)

In [13]:
trainingSet = data.build_full_trainset()

In [14]:
knn.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [15]:
testSet = trainingSet.build_anti_testset()

In [16]:
predictions = knn.test(testSet)

In [17]:
predictions[0:4]

[Prediction(uid=1395430, iid=12, r_ui=3.5875392576478844, est=3.6747462221288183, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1395430, iid=30, r_ui=3.5875392576478844, est=3.2755272641840896, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1395430, iid=44, r_ui=3.5875392576478844, est=3.374687387588717, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1395430, iid=47, r_ui=3.5875392576478844, est=3.974992667401253, details={'actual_k': 40, 'was_impossible': False})]

In [18]:
titles = pd.read_csv('movie_titles.csv', encoding = "ISO-8859-1", 
                     header = None, 
                     names = ['Movie_Id', 'Year', 'Name'])

In [19]:
from collections import defaultdict

In [20]:
def get_top3_recommendations(predictions, topN = 3):
     
    top_recs = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_recs[uid].append((iid, est))
     
    for uid, user_ratings in top_recs.items():
        user_ratings.sort(key = lambda x: x[1], reverse = True)
        top_recs[uid] = user_ratings[:topN]
     
    return top_recs

In [21]:
top3_recommendations = get_top3_recommendations(predictions)

In [22]:
top3_recommendations

defaultdict(list,
            {1395430: [(5577, 4.226298013573388),
              (12721, 4.201988201339796),
              (12478, 4.125322522471735)],
             66358: [(1367, 3.9750732007157907),
              (12500, 3.9501726288056926),
              (16343, 3.92536403369447)],
             1947516: [(13593, 4.850158816133738),
              (16711, 4.82513023431578),
              (12530, 4.800080929508842)],
             306466: [(4877, 4.375173611207626),
              (11166, 4.349655698784912),
              (5843, 4.326043881400433)],
             1987434: [(15296, 4.625817721557596),
              (16223, 4.600340978197035),
              (6739, 4.600201540754212)],
             2175886: [(15777, 4.250228715991721),
              (16415, 4.250104213137232),
              (15062, 4.224703399379016)],
             2459251: [(2532, 4.525511682192603),
              (17335, 4.524056429321666),
              (4877, 4.501160827316804)],
             369822: [(9430, 3.100758481

In [23]:
def print_recs(i):
    for (a, b) in top3_recommendations[i]:
        print(titles[titles.Movie_Id == a]['Name'].values[0], np.round(b,2))

In [24]:
i = 262149
print_recs(i)

In [25]:
i = np.random.choice(list(top3_recommendations.keys()))
print(i)
print_recs(i)

1863803
Jackie Brown: Collector's Series 3.3
Sex and the City: Season 6: Part 2 3.25
Six Feet Under: Season 4 3.25


In [26]:
films = data.df[(data.df.Cust_Id == i) & (data.df.Rating == 5)]['Movie_Id'].values
titles[titles.Movie_Id.isin(films)]['Name'].values

array(['Sex and the City: Season 4', 'Bottle Rocket', 'Rushmore',
       'To Die For', 'The Sopranos: Season 3', 'Six Feet Under: Season 1',
       'Annie Hall', 'Deadwood: Season 1', 'The Royal Tenenbaums',
       'Ghost World', 'Pulp Fiction', 'Sex and the City: Season 5',
       'Lost in Translation', 'Rescue Me: Season 1',
       'The Shawshank Redemption: Special Edition',
       'The Sopranos: Season 1'], dtype=object)